# Labelling Algorithm
The intention of this notebook is, to learn how to use C Code in Python and with that knowledge implement a labelling algorithm for the CVRP in C.

## CFFI Tutorial
https://medium.com/@torbengraebergt/c-c-and-python-class-interfacing-f45a9ee352f4

In [1]:
from cffi import FFI

dllURI = "labelling_lib.so"

ffi = FFI()
labelling_lib = ffi.dlopen(dllURI)

funDefs = "void initGraph(unsigned num_nodes, unsigned* node_data, double* edge_data, const double capacity); unsigned int myProcessorInit(int size, double* data); int myProcessorProcess(unsigned int id, double *d, int size); void labelling(double const * dual,const bool farkas, unsigned* result);"
ffi.cdef(funDefs, override=True)

In [2]:
class myProcessor():

    def __init__(self, size, data):
        pointer = ffi.cast("double*",data.ctypes.data)
        self.id = labelling_lib.myProcessorInit(size, pointer)

    def process(self, data, size):
        data = ensure_np_type(data, 'float64')
        pointer_to_data = get_pointer_to_np_arr(data, "double*", ffi)
        
        ret = labelling_lib.myProcessorProcess(self.id, pointer_to_data, size)
        if ret<0:
            raise ValueError("Sth bad happened in c++ code")
        else:
            return data

In [3]:
import numpy as np

def ensure_np_type(arr, target_type):
    if arr.dtype is not np.dtype(target_type):
        return arr.astype(np.dtype(target_type))
    else:
        return arr

def get_pointer_to_np_arr(arr, ctype, ffi):
    return ffi.cast(ctype, arr.ctypes.data)

In [4]:
data = np.arange(10)
processor = myProcessor(10,data)
processor.process(data,10)

array([ 0.,  1.,  4.,  9., 16., 25., 36., 49., 64., 81.])

## Passing a Graph to the C Code

In [5]:
import networkx as nx
import random
import numpy as np

In [15]:
G = nx.complete_graph(10)
for (u,v) in G.edges():
    G.edges[u,v]['weight'] = random.randint(1,10)
    
# G.edges()[7,9]['weight'] = 100
for node in G.nodes():
    G.nodes()[node]['demand'] = random.randint(1,10)

In [16]:
node_data = list(nx.get_node_attributes(G,"demand").values())

nodes_arr = ffi.cast("unsigned*", np.array(node_data).astype(np.uintc).ctypes.data)
# TODO: Extract raw matrix from G and pass it to initGraph
edges = nx.adjacency_matrix(G,dtype="float64").toarray()
edges_arr = ffi.cast("double*", edges.ctypes.data)
num_nodes = ffi.cast("unsigned",G.number_of_nodes())

capacity = 30
capacity_ptr = ffi.cast("double",capacity)

result = np.zeros(capacity,dtype=np.uintc)
result_arr = ffi.cast("unsigned*",result.ctypes.data)



/var/folders/tv/pzvp6rwx029b6p4rgqv2cx0h0000gn/T/ipykernel_8307/729666234.py:5: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  edges = nx.adjacency_matrix(G,dtype="float64").toarray()


In [ ]:
labelling_lib.initGraph(num_nodes,nodes_arr,edges_arr, capacity_ptr)

dual = np.array(range(10)).astype(np.float64)
dual = np.full(10,4,dtype=np.double)
pointer_dual = ffi.cast("double*", dual.ctypes.data)

labelling_lib.labelling(pointer_dual, True,result_arr)

print(result)
